In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import bwsample as bws

# Toy Data

In [3]:
data = (
    ([1, 0, 0, 2], ['A', 'B', 'C', 'D']),
    ([1, 0, 0, 2], ['A', 'B', 'C', 'D']), 
    ([2, 0, 0, 1], ['A', 'B', 'C', 'D']), 
    ([0, 1, 2, 0], ['A', 'B', 'C', 'D']),
    ([0, 1, 0, 2], ['A', 'B', 'C', 'D']),
)

"""
data = (
    ([1, 0, 0, 2], ['A', 'B', 'C', 'D']),
    ([1, 0, 0, 2], ['A', 'B', 'C', 'D']), 
    ([2, 0, 0, 1], ['A', 'B', 'C', 'D']), 
    ([1, 2, 0, 0], ['D', 'E', 'F', 'A']),
    ([0, 2, 1, 0], ['D', 'E', 'F', 'A']),
    ([0, 0, 1, 2], ['D', 'E', 'F', 'A'])
)
"""

# Extract pair frequencies
dok, _, _, _ = bws.extract_pairs_batch2(data)

In [4]:
# convert to sparse matrix
cnt, _ = bws.to_scipy(dok)
print(cnt.todense().astype(int))

[[0 2 3 3]
 [3 0 2 4]
 [1 0 0 3]
 [1 1 2 0]]


# Simple Ratios
We can compute the ratios

$$
\mu_{ij} = \frac{N_{ij}}{N_{ij} + N_{ji}} \; \forall i,j
$$

with the matrix operations

$$
\textbf{M} = \frac{\textbf{N}}{\textbf{N} + \textbf{N}^T}
$$

In [5]:
ranked, ordids, scores, ratios = bws.rank(dok, method='ratio')

print(ratios.todense().round(3), "\n")
print(f"positions: {ranked}") 
print(f"ordered IDs: {ordids}") 
print(f"scores: {scores}") 

[[0.   0.4  0.75 0.75]
 [0.6  0.   1.   0.8 ]
 [0.25 0.   0.   0.6 ]
 [0.25 0.2  0.4  0.  ]] 

positions: [1, 0, 2, 3]
ordered IDs: ['B', 'A', 'C', 'D']
scores: [1.0, 0.6774193548387094, 0.0, 0.0]


# p-values based on Chi-Squared test
The question which opposing frequency $N_{ij}$ or $N_{ji}$ is larger,
can be treated as hypothesis test:

$$
\mu = \frac{N_{ij}}{N_{ij} + N_{ji}}
\\
H_0: \mu = 0.5
\\
H_a: \mu > 0.5
$$

We basically want to run [binomal test](https://en.wikipedia.org/wiki/Binomial_test) for an coin tossing type of experiment. However, we must expect larger values of $N$, and thus the [Pearson's $\chi^2$-test](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test) is more favorable due to its continuous distribution.

When $N_{ij} > N_{ji}$,
the lower the p-value of the Pearson Chi-Squared test, 
the more significant is the rejection of H0.
The function `bwsample.scale_pvalues` computes the $(1-p)$ value if $N_{ij} > N_{ji}$, 
or returns 0.0 otherwise.

$$
x_{ij} = 
\left \{
\begin{aligned}
& 1-p, & \text{if} \, N_{ij} > N_{ji} \\
& 0, & \text{otherwise}
\end{aligned} 
\right.
\quad
\forall i,j
$$

In other words, the higher $x_{ij}$, the more signification is the $N_{ij} > N_{ji}$

In [6]:
ranked, ordids, scores, minuspvalue = bws.rank(dok, method='pvalue')

print(minuspvalue.todense().round(3), "\n")
print(f"positions: {ranked}") 
print(f"ordered IDs: {ordids}") 
print(f"scores: {scores}") 

[[0.    0.    0.683 0.683]
 [0.345 0.    0.843 0.82 ]
 [0.    0.    0.    0.345]
 [0.    0.    0.    0.   ]] 

positions: [1, 0, 2, 3]
ordered IDs: ['B', 'A', 'C', 'D']
scores: [1.0, 0.679879058378886, 0.17192887014555752, 0.0]


### Notes
Why should we care about the size of $N$?
A small gap between $N_{ij}=1$ vs $N_{ji}=2$ might be interpretated big ratio gap $1/3$ vs $2/3$.
However, when collecting more data it might turn out $N_{ij}=1000$ vs $N_{ji}=1001$

In [7]:
import scipy.stats

for f1 in (1, 10, 100, 1000):
    f2 = f1 + 1
    fe = (f1 + f2) / 2
    _, pval = scipy.stats.chisquare(f_obs=[f1, f2], f_exp=[fe, fe], ddof=0)
    print(f"p-value={pval:5.3f} | Nij={f1} vs Nji={f2}")

p-value=0.564 | Nij=1 vs Nji=2
p-value=0.827 | Nij=10 vs Nji=11
p-value=0.944 | Nij=100 vs Nji=101
p-value=0.982 | Nij=1000 vs Nji=1001


# Eigenvector Solution as Scores
[Saaty (2003)](http://dx.doi.org/10.1016/S0377-2217(02)00227-8) derives the scores from pairwise comparision data by solving a eigenvalue problem.

We compute a ratio matrix $A=(a_{ij})$

$$
a_{ij} = 
\left \{
\begin{aligned}
& N_{ij} / N_{ji}, & \text{if} \, N_{ji} > 0 \\
& 0, & \text{otherwise}
\end{aligned} 
\right.
\quad
\forall i,j
$$

Given the scalar $m$ the eigenvalue, the scores $s_i$ the eigenvector, 
we can solve equation

$$
A s = m s
$$


In [8]:
ranked, ordids, scores, _ = bws.rank(dok, method='eigen')

print(f"positions: {ranked}") 
print(f"ordered IDs: {ordids}") 
print(f"scores: {scores}") 

positions: [1, 0, 3, 2]
ordered IDs: ['B', 'A', 'D', 'C']
scores: [0.6981531725668787, 0.6551014935159343, 0.20482709915914166, 0.2036419413530658]


# Simulate Transitions
We compute a transition probability matrix $\Pr(k|j)$ of items being evaluated $e_k > e_j$.
Assume our initial items are equally distributed with item probability $\pi_j = 1/N \; \forall j$,
then we can predict $\pi_k = \pi_j \cdot \Pr(k|j)$.
The most probable item $k^*$ to be evaluated $e_{k^*} > e_j$
is $k^* = \arg\max(\pi_k)$.


In [9]:
ranked, ordids, scores, (x, transmat) = bws.rank(dok, method='transition', n_rounds=3)

print(transmat.todense().round(3))
print(f"predicted probabilities: {x}", "\n") 

print(f"positions: {ranked}") 
print(f"ordered IDs: {ordids}") 
print(f"scores: {scores}") 

[[0.488 0.278 0.098 0.136]
 [0.299 0.484 0.05  0.167]
 [0.237 0.197 0.409 0.156]
 [0.207 0.224 0.134 0.435]]
predicted probabilities: [0.33187441 0.31953262 0.13568109 0.21291187] 

positions: [0, 1, 3, 2]
ordered IDs: ['A', 'B', 'D', 'C']
scores: [1.0, 0.9370937652080424, 0.39364634516069397, 0.0]
